# Automate Rules that pin top selling items & Boost Top Categories

## Requirments

* Will need to have an index with records that include category. 
* You will need to create an export of categories using browse end-point
* Will need to have historical analytics data that has the following fields. 
    1. query
    2. record
    3. position (rank)
* If you don't want to boost category then you can skip the part where we import category. 

In [10]:
#Set all the necessary configs
#API_KEY has to be admin/write

INDEX_NAME = "Variants"
APP_ID = ""
API_KEY = ""
ANALYTICS_FILE = "boost.csv"
CATEGORIES_FILE = "category_export.json"
BATCH_SIZE = 5000


In [2]:
# import and initiate algolia index
from algoliasearch.search_client import SearchClient
import csv

client = SearchClient.create(APP_ID, API_KEY)


# Create a new index and add a record
index = client.init_index(INDEX_NAME)


In [4]:
# this function will check to see if a string contains special characters.
# we need to do this in-order to make sure that the queries we add to rules are valid
import re

# Function checks if the string
# contains any special character

def run(input_string):
    pattern = r'^[a-zA-Z0-9\s]+$'
    return bool(re.match(pattern, input_string))


### Analytics File Breakdown

The analytics file export will look something like this: 

| query | Record | position |
------------------------------
| honey | obj123 | 1 |
| honey | obj124 | 2 |
| honey | obj125 | 3 |
| milk | obj126 | 1 |
| milk | obj127 | 2 |
| milk | obj128 | 3 |
-----------------------------

The goal will be to create a rule with the following config

Rule 1: 
condition: 
    query: honey
consequence:
    pin: 
        obj123: 1
        obj124: 2
        obj125: 3
    boost: 
        honey category

Rule 2: 
condition: 
    query: milk
consequence:
    pin: 
        obj126: 1
        obj127: 2
        obj128: 3
    boost: 
        milk category


In [3]:
# read in the analytics file and add the records into an array
list_of_terms = []
with open(ANALYTICS_FILE, "r") as data:
    new_data = csv.DictReader(data, delimiter=',')
    for d in new_data:
        list_of_terms.append(d)

In [6]:
# this next function will loop through the list_of_terms
# and create an object where the key is the query and value is an object with the necessary data for the rule

# SKU = objectId
# Suggested Rank = Position

# Region is commented out because in my test case region was a secondary condition. 

proper_list = {}
for term_obj in list_of_terms:
    sku = term_obj["SKU"]
    query = term_obj["Query"]
    if not run(query):
        continue
    #region = term_obj["Region"]
    #query_region = query + "_" + region
    new_obj = {
        "query": query,
        "sku": sku,
        "position": term_obj["Suggested Rank"],
   ##     "region": region
    }
    if query not in proper_list:
        proper_list[query] = [new_obj]
    else:
        proper_list[query].append(new_obj)

In [7]:
# The next function will create the rules object and add them to an array called list_of_rules
# I use the hash function to create the objectID for the rule. 

list_of_rules = []
for key, value in proper_list.items():
    rule_obj = {}
    rule_obj["objectID"] = str(hash(key))
    rule_obj["conditions"] = [
        {
            "pattern": key,
            "anchoring": 'is',
            "alternatives": True
        
        },
    ]
    consequences = []
    for skus in value:
        promote = {
            "objectID": skus["sku"],
            "position": int(skus["position"]) - 1
        }
        if any(obj["objectID"] == skus["sku"] for obj in consequences):
            continue
        consequences.append(promote)
    rule_obj["consequence"] = {"promote": consequences, "filterPromotes": True}
    list_of_rules.append(rule_obj)

In [11]:
# We will now import the categories export
# if we don't want to boost categories you can skip the next two. 

import json
with open(CATEGORIES_FILE, "r") as import_file:
    categories = json.load(import_file)


In [12]:
# this will take the list of categories and the list of rules
# find the category that belongs to the top pinned item
# once it does that it will add it as consequence. optionalFilters in rules need to be enabled for the App. 
# this might take some time to run. Not fully optimized

def get_lowest_position_category(objects_list, categories_list):
    for obj in objects_list:
        lowest_position = float('inf')
        lowest_position_category = None
        promote_list = obj.get("consequence", {}).get("promote", [])
        for promote_item in promote_list:
            position = promote_item.get("position", 0)
            if position < lowest_position:
                lowest_position = position
                object_id = promote_item.get("objectID")
                for category in categories_list:
                    if category.get("objectID") == object_id:
                        lowest_position_category = category.get("Category")
                        break
        if lowest_position_category:
            obj["consequence"]["params"] = {
                "optionalFilters": "Category:"+lowest_position_category}
    return objects_list

# run the function
list_of_rules = get_lowest_position_category(list_of_rules, categories)


In [13]:
# create batches to send the rules to algolia. This will allow us to use a try / catch to monitor sends.
# you can either increase/decrease the batch size

def create_batches(input_array):
    batch_size = BATCH_SIZE
    num_batches = len(input_array) // batch_size
    remainder = len(input_array) % batch_size

    batches = [input_array[i*batch_size:(i+1)*batch_size]
               for i in range(num_batches)]
    if remainder > 0:
        batches.append(input_array[num_batches*batch_size:])

    return batches


In [14]:
# Run the create function
batches = create_batches(list_of_rules)

In [ ]:
# This will save the rules into your algolia index
for batch in batches:
    try:
        index.save_rules(batch)
    except Exception as error:
        print(error)